In [1]:
from __future__ import division

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import random

%matplotlib inline

In [2]:
data = np.loadtxt("synthetic_control.txt")
# target = data[:,0]
# data = data[:,1:]

# print(data[0:10])

data = data / np.linalg.norm(data)

In [3]:
data[0]

array([0.00476076, 0.00570063, 0.0051837 , 0.00517465, 0.00478383,
       0.00558424, 0.00420095, 0.00459596, 0.00583043, 0.00448529,
       0.00543737, 0.00483286, 0.00595902, 0.00534893, 0.00571083,
       0.00543737, 0.00564341, 0.0043873 , 0.00457568, 0.0043618 ,
       0.00426339, 0.00484161, 0.00508354, 0.00488055, 0.00546343,
       0.00414192, 0.00478317, 0.00402674, 0.00432061, 0.00577989,
       0.00414015, 0.0044051 , 0.00589762, 0.00470354, 0.00482168,
       0.00465774, 0.00433259, 0.00551123, 0.005124  , 0.00447345,
       0.00587782, 0.00433963, 0.00479635, 0.00529378, 0.005137  ,
       0.00566624, 0.00464346, 0.00478706, 0.00587168, 0.00492051,
       0.00519944, 0.00406179, 0.00558151, 0.00414301, 0.00577814,
       0.00578742, 0.00537127, 0.00552077, 0.00421225, 0.00427949])

In [4]:
def compete(input, net, n_features):
    bmu_idx = np.array([0, 0])
    idx_x = -1
    idx_y = -1
    min_dist = np.iinfo(np.int).max
    
    # calculate the distance between each neuron and the input
    for x in range(net.shape[0]):
        for y in range(net.shape[1]):
            w = net[x, y, :].reshape(n_features, 1)
            sq_dist = np.sum((w - input) ** 2)
            sq_dist = np.sqrt(sq_dist)
            if sq_dist < min_dist:
                min_dist = sq_dist # dist
#                 bmu_idx = np.array([x, y]) # id
                idx_x = x
                idx_y = y
    
    bmu = net[bmu_idx[0], bmu_idx[1], :].reshape(n_features, 1)
#     return (bmu, bmu_idx)
    return bmu, idx_x, idx_y

In [5]:
def decay_param(param_value, t, constant):
    return param_value * np.exp(-t / constant)

def decay_radius(initial_radius, i, time_constant):
    return initial_radius * np.exp(-i / time_constant)

def decay_learning_rate(initial_learning_rate, i, n_iterations):
    return initial_learning_rate * np.exp(-i / n_iterations)

def calculate_influence(distance, radius):
    return np.exp(-distance / (2* (radius**2)))

In [6]:
n_iterations = 300
init_lr = 0.8

rows = 10
cols = 60
n_features = data.shape[1]
n_elements = data.shape[0]

init_radius = max(rows, cols) / 2
time_constant = n_iterations / np.log(init_radius)

In [7]:
net = np.random.random((rows, cols, n_features))

In [8]:
idxs = np.arange(600)
np.random.shuffle(idxs)

In [ ]:
for i in range(0, n_iterations):#, len(idxs)):
#     r = decay_param(init_radius, i, time_constant)
#     l = decay_param(init_lr, i, time_constant)
    
    r = decay_radius(init_radius, i, time_constant)
    l = decay_learning_rate(init_lr, i, n_iterations)    
#     example = random.randint(0,177)
#     t = (data[example])
#     bmu, idx_x, idx_y = compete(t, net, n_features)    
#     t = np.matrix(t).T        
#     for x in range(net.shape[0]):
#         for y in range(net.shape[1]):
#             w = net[x, y, :].reshape(n_features, 1)
#             aux = ((np.array([x, y]) - np.array([idx_x, idx_y])) ** 2)
#             w_dist = np.sqrt(np.sum(aux))

#             if w_dist <= r: #Update weghts from neit...
#                 # calculate the degree of influence (based on the 2-D distance)
#                 influence = calculate_influence(w_dist, r)                
#                 new_w = w + (l * influence * (t - w))
#                 net[x, y, :] = new_w.reshape(n_features)
             
    # select a training example at random
    for j, example in enumerate(idxs): 
#         r = decay_radius(init_radius, i, time_constant)
#         l = decay_learning_rate(init_lr, i, n_iterations)
        t = (data[example])
        # find its Best Matching Unit
        bmu, idx_x, idx_y = compete(t, net, n_features)        
        # update weight vector to move closer to input
        # and move its neighbours in 2-D vector space closer        
        t = np.matrix(t).T        
        for x in range(net.shape[0]):
            for y in range(net.shape[1]):
                w = net[x, y, :].reshape(n_features, 1)
                aux = ((np.array([x, y]) - np.array([idx_x, idx_y])) ** 2)
                w_dist = np.sqrt(np.sum(aux))
    
                if w_dist <= r: #Update weghts from neit...                    
                    influence = calculate_influence(w_dist, r)                    
                    new_w = w + (l * influence * (t - w))
                    net[x, y, :] = new_w.reshape(n_features)

    print(r, i)
    

30.0 0
29.661801020165676 1


In [ ]:
center_1 = np.mean(data[0:100], axis = 0)
center_2 = np.mean(data[100:200], axis = 0)
center_3 = np.mean(data[200:300], axis = 0)
center_4 = np.mean(data[300:400], axis = 0)
center_5 = np.mean(data[400:500], axis = 0)
center_6 = np.mean(data[500:600], axis = 0)


In [ ]:
print(net.shape)
fig = plt.figure()

ax = fig.add_subplot(111, aspect='equal')
ax.set_xlim((0, net.shape[0]+1))
ax.set_ylim((0, net.shape[1]+1))
ax.set_title('Self-Organising Map after %d iterations' % n_iterations)

# plot
for x in range(1, net.shape[0] + 1):
    for y in range(1, net.shape[1] + 1):
#         print('net: ', net[x-1,y-1,:])
        element = net[x-1,y-1,:]
#         print(element[0:4])
        aux_r = np.average(element[0:4])
        aux_g = np.average(element[4:8])
        aux_b = np.average(element[8:])
#         print(aux_r, aux_g, aux_b)
#         aux = np.sum(element)
#         print(aux_r)
#         print(aux_g)
#         print(aux_b)
        
        ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1, facecolor = [aux_r,aux_g,aux_b], edgecolor='none'))
#         ax.add_patch(patches.Rectangle((x-0.5, y-0.5), 1, 1, color = [(aux_r,aux_g,aux_b)]
#                      facecolor=[aux_r, aux_g, aux_b]))
plt.show()

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# X     = np.array([[1, 2], [1, 4], [1, 0],[10, 2], [10, 4], [10, 0]])
kmeans = KMeans(n_clusters=3, random_state=0).fit(data)
kmeans.cluster_centers_

In [ ]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []

# clusters = kmeans.cluster_centers_

clusters = np.array([center_1,center_2,center_3,center_4,center_5,center_6])
for x in range(net.shape[0]):
    for y in range(net.shape[1]):
        minimo  = np.inf
        idx_min = 9
        for idx, center in enumerate(clusters):            
            aux = ((net[x, y, :] - center)** 2)
            w_dist = np.sqrt(np.sum(aux))
            if w_dist < minimo:
                minimo  = w_dist
                idx_min = idx
        
        if idx_min == 0:
            list_1.append((x, y))
        elif idx_min == 1:
            list_2.append((x, y))
        elif idx_min == 2:
            list_3.append((x, y))
        elif idx_min == 3:
            list_3.append((x, y))
        elif idx_min == 4:
            list_3.append((x, y))
        elif idx_min == 5:
            list_3.append((x, y))
        elif idx_min == 6:
            list_3.append((x, y))

global_list = []
global_list.append(list_1)
global_list.append(list_2)
global_list.append(list_3)
global_list.append(list_4)
global_list.append(list_5)
global_list.append(list_6)
            

In [ ]:
fig = plt.figure()

ax = fig.add_subplot(111, aspect='equal')
ax.set_xlim((0, net.shape[0]))
ax.set_ylim((0, net.shape[1]))
ax.set_title('Self-Organising Map after %d iterations' % n_iterations)

for idx, lista in enumerate(global_list):
    for item in lista:
        x = item[0]
        y = item[1]
        
        if idx == 0:
            ax.add_patch(patches.Rectangle((x, y), 1, 1, facecolor = [1,0,0], edgecolor='none'))
        elif idx == 1:
            ax.add_patch(patches.Rectangle((x, y), 1, 1, facecolor = [0,0.8,0], edgecolor='none'))
        elif idx == 2:
            ax.add_patch(patches.Rectangle((x, y), 1, 1, facecolor = [0,0,1], edgecolor='none'))                                
plt.show()